In [1]:
import os
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport
import re
import matplotlib.pyplot as plt


In [2]:
_data_dir = os.path.join('/Users/murraywebb/Projects/sightly', 'raw')

In [3]:
def load_data(filename):
    #dtype = {
    #    'cpv': np.int32,
    #}
    dates = ['StartDate','EndDate']
    if filename == 'allOrders.csv': 
        return pd.read_csv(os.path.join(_data_dir, filename), parse_dates=dates, sep=',', engine='c')
    else:
        return pd.read_csv(os.path.join(_data_dir, filename), sep=',', engine='c')



In [4]:
allOrders = load_data('allOrders.csv')

#budget is client selected budget across flight time
#sightlySpend is cumSum since startDate...

In [5]:
allOrders.dtypes

AccountName               object
OrderId                   object
StartDate         datetime64[ns]
EndDate           datetime64[ns]
CampaignBudget           float64
SightlySpend             float64
CPV                      float64
dtype: object

In [6]:
allOrders.describe(include='all')

,AccountName,OrderId,StartDate,EndDate,CampaignBudget,SightlySpend,CPV
count,543,543,543,543,543.000000,543.000000,543.000000
unique,11,527,265,208,NaN,NaN,NaN
top,AMG,A4420CD5-0A23-4C2E-9580-36B53BFA6AF8,2017-12-01 00:00:00,2017-12-31 00:00:00,NaN,NaN,NaN
freq,168,3,11,29,NaN,NaN,NaN
first,NaN,NaN,2017-01-17 00:00:00,2017-04-11 00:00:00,NaN,NaN,NaN
last,NaN,NaN,2019-09-01 00:00:00,2019-10-21 00:00:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,6587.975635,3122.769412,0.003297
std,NaN,NaN,NaN,NaN,8231.770955,4150.225126,0.001179
min,NaN,NaN,NaN,NaN,300.000000,160.875000,0.001500
25%,NaN,NaN,NaN,NaN,1500.000000,676.500000,0.002500


In [7]:
advertiserCategories = load_data('advertiserCategories.csv')
advertiserCategories.AdvertiserSubCategory.value_counts()

Car Dealers                              94
Colleges & Universities                  17
Jewelry                                  17
Casinos                                  14
Educational Services                     11
Home & Garden: Appliances                10
Ticket Sales                             10
Car Dealers - New                         9
Banks & Credit Unions                     8
Museums                                   7
Motorcycle Dealers                        7
Performing Arts                           5
Motorsport Vehicle Dealers                5
Windows Installation                      4
Opera & Ballet                            4
Residential Developers                    4
Lawyers: Personal Injury Law              4
Parks                                     3
Lawyers                                   3
Middle Schools & High Schools             3
Beaches                                   3
Travel Services: Visitor Centers          3
Medical Centers                 

In [8]:
objectives = load_data('objectives.csv')
objectives.head()

,AccountName,OrderId,Objective
0,AMG,00000000-0000-0000-0000-000000000000,Awareness & Reach
1,AMG,F1A8BEE4-E7FD-4597-8D29-000F74CFAE24,Full Funnel
2,MMG,96044EBB-2176-481F-BEAF-006B52B26944,Full Funnel
3,MMG,F8906728-5714-42C1-8121-040457CA96D5,Full Funnel
4,AMG,8537F908-0430-4991-9001-04A95CBDD6C5,Full Funnel


In [9]:
targetedAgeRanges = load_data('targetedAgeRanges.csv')
targetedAgeRanges.head()

,AccountName,OrderId,AgeGroup
0,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,Undetermined
1,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,25-34
2,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,35-44
3,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,55-64
4,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,45-54


In [10]:
targetedDevices = load_data('targetedDevices.csv')
targetedDevices.head()

,AccountName,OrderId,Device
0,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,Mobile devices with full browsers
1,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,Computers
2,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,Other
3,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,Tablets with full browsers
4,AMG,F1A8BEE4-E7FD-4597-8D29-000F74CFAE24,NaN


In [11]:
targetedGenders = load_data('targetedGenders.csv')
targetedGenders.head()

,AccountName,OrderId,Gender
0,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,Undetermined
1,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,Male
2,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,Female
3,AMG,F1A8BEE4-E7FD-4597-8D29-000F74CFAE24,Female
4,AMG,F1A8BEE4-E7FD-4597-8D29-000F74CFAE24,Male


In [12]:
targetedGeos = load_data('targetedGeos.csv')
targetedGeos.head(10)

,AccountName,OrderId,Geo
0,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,"Detroit, Michigan DMA"
1,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,"Toledo, Ohio DMA"
2,AMG,F1A8BEE4-E7FD-4597-8D29-000F74CFAE24,"Nashville, Tennessee DMA"
3,AMG,2C011161-E3C8-4165-8619-17D676321B98,"Atlanta, Georgia DMA"
4,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"36587, Alabama"
5,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"36608, Alabama"
6,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"39466, Mississippi"
7,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"39503, Mississippi"
8,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"39532, Mississippi"
9,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"39564, Mississippi"


In [23]:
targetedGeos.Geo.value_counts()

Birmingham-Anniston-Tuscaloosa, Alabama DMA    35
01585, Massachusetts                           26
01069, Massachusetts                           25
01109, Massachusetts                           24
01453, Massachusetts                           24
01079, Massachusetts                           24
01080, Massachusetts                           23
01118, Massachusetts                           23
01057, Massachusetts                           23
15025, Pennsylvania                            22
01010, Massachusetts                           22
01092, Massachusetts                           22
01119, Massachusetts                           22
01056, Massachusetts                           22
01028, Massachusetts                           22
01009, Massachusetts                           22
01083, Massachusetts                           22
01082, Massachusetts                           21
17315, Pennsylvania                            21
01095, Massachusetts                           21


In [12]:
targetedHHIs = load_data('targetedHHIs.csv')
targetedHHIs.head()

,AccountName,OrderId,HHI
0,Advance Local,6BA9AF79-3673-40F9-A968-A8B0016517FD,11 - 20%
1,Advance Local,6BA9AF79-3673-40F9-A968-A8B0016517FD,21 - 30%
2,Advance Local,6BA9AF79-3673-40F9-A968-A8B0016517FD,31 - 40%
3,Advance Local,6BA9AF79-3673-40F9-A968-A8B0016517FD,41 - 50%
4,Advance Local,6BA9AF79-3673-40F9-A968-A8B0016517FD,Lower 50%


In [13]:
targetedParentalStatuses = load_data('targetedParentalStatuses.csv')
targetedParentalStatuses.head()

,AccountName,OrderId,ParentalStatus
0,Advance Local,6BA9AF79-3673-40F9-A968-A8B0016517FD,Not
1,Advance Local,6BA9AF79-3673-40F9-A968-A8B0016517FD,Parent
2,Advance Local,6BA9AF79-3673-40F9-A968-A8B0016517FD,Unknown
3,Advance Local,F5A9CCB2-991B-4508-A961-A8B001656D2C,Not
4,Advance Local,F5A9CCB2-991B-4508-A961-A8B001656D2C,Parent


In [16]:
ProfileReport(allOrders, check_correlation=False)

Number of variables,7
Number of observations,543
Total Missing (%),0.0%
Total size in memory,29.8 KiB
Average record size in memory,56.1 B
Numeric,3
Categorical,2
Boolean,0
Date,2
Text (Unique),0
Rejected,0


In [84]:
profile = ProfileReport(allOrders)
profile.to_file(outputfile="/Users/murraywebb/Projects/sightly/all_orders_profiler.html")

In [15]:
#allOrdersCats = allOrders.merge(targetedGeos, on='OrderId', how = 'left')

In [ ]:
#allOrdersCats.describe()

In [ ]:
#allOrdersCats.head()

In [114]:
#multiple join - make sure to check for validity...this will produce sparse output - 
###this is the full data (all csv's) joined...starting much smaller and working up
dfs = [allOrders, objectives, advertiserCategories, targetedDevices, targetedAgeRanges, targetedGenders, targetedGeos, targetedHHIs
      ,targetedParentalStatuses]
dfs = [df.set_index(['AccountName','OrderId']) for df in dfs]
allData = dfs[0].join(dfs[1:])

In [117]:
#allData.describe(include='all')
allData.dtypes

StartDate                datetime64[ns]
EndDate                  datetime64[ns]
CampaignBudget                  float64
SightlySpend                    float64
CPV                             float64
Objective                        object
AdvertiserSubCategory            object
Device                           object
AgeGroup                         object
Gender                           object
Geo                              object
HHI                              object
ParentalStatus                   object
dtype: object

In [123]:
profile = ProfileReport(allData, check_correlation=False)
profile.to_file(outputfile="/Users/murraywebb/Projects/sightly/all_data_profiler.html")

In [155]:
#
allData.Objective.value_counts()


Full Funnel    26849753
Name: Objective, dtype: int64

In [156]:
### Data notes:
#1. 527 order IDs, I am trying to predict CPV at order ID level. 
#2. Geo is messy and not structured, must break it down into a reasonable hiercarchy and then treat it in a categorical manner
#3. 

In [20]:
#ProfileReport(allData) 

Number of variables,15
Number of observations,29396903
Total Missing (%),4.6%
Total size in memory,3.3 GiB
Average record size in memory,120.0 B
Numeric,2
Categorical,12
Boolean,0
Date,1
Text (Unique),0
Rejected,0


In [10]:
targetedGeos.head(10)
#county contains 'county'
#zip is numeric 
#DMA
#state 

,AccountName,OrderId,Geo
0,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,"Detroit, Michigan DMA"
1,Advance Local,F9A38330-9367-47A4-83AB-A8CC0153987D,"Toledo, Ohio DMA"
2,AMG,F1A8BEE4-E7FD-4597-8D29-000F74CFAE24,"Nashville, Tennessee DMA"
3,AMG,2C011161-E3C8-4165-8619-17D676321B98,"Atlanta, Georgia DMA"
4,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"36587, Alabama"
5,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"36608, Alabama"
6,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"39466, Mississippi"
7,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"39503, Mississippi"
8,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"39532, Mississippi"
9,AMG,65DAE8B5-AC59-40DF-B3DA-23A269EC7661,"39564, Mississippi"


In [78]:
sum(targetedGeos.Geo.str.contains('DMA')) #non-unique

287

In [7]:
targetedGeos['Geo'][:10]

0       Detroit, Michigan DMA
1            Toledo, Ohio DMA
2    Nashville, Tennessee DMA
3        Atlanta, Georgia DMA
4              36587, Alabama
5              36608, Alabama
6          39466, Mississippi
7          39503, Mississippi
8          39532, Mississippi
9          39564, Mississippi
Name: Geo, dtype: object

In [9]:
regex = (r'(?P<city>[A-Za-z ]+), '      
 r'(?P<state>[A-Z]{2}) '       
r'(?P<zip>\d{5}(?:-\d{4})?)')  
#.str.extract(regex)

In [15]:
regex_state = r'(^([^,]+),([^,]+))'
geo = targetedGeos.Geo.str.replace('DMA','').str.extract(regex_state)

In [28]:
geo.head()

,0,1,2
0,"Detroit, Michigan",Detroit,Michigan
1,"Toledo, Ohio",Toledo,Ohio
2,"Nashville, Tennessee",Nashville,Tennessee
3,"Atlanta, Georgia",Atlanta,Georgia
4,"36587, Alabama",36587,Alabama


In [31]:
geo.columns=['full_geo', 'first_position', 'state']

In [38]:
geo.head(20)

,full_geo,first_position,state
0,"Detroit, Michigan",Detroit,Michigan
1,"Toledo, Ohio",Toledo,Ohio
2,"Nashville, Tennessee",Nashville,Tennessee
3,"Atlanta, Georgia",Atlanta,Georgia
4,"36587, Alabama",36587,Alabama
5,"36608, Alabama",36608,Alabama
6,"39466, Mississippi",39466,Mississippi
7,"39503, Mississippi",39503,Mississippi
8,"39532, Mississippi",39532,Mississippi
9,"39564, Mississippi",39564,Mississippi


In [39]:
geo['zip'] = geo.first_position.str.extract(regex_zip)

In [33]:
regex_city_county = r'(?P<city_county>[-A-Za-z ]+), '
regex_zip = r'(?P<zip>\d{5}(?:-\d{4})?)'
zips = targetedGeos.Geo.str.replace('DMA','').str.extract(regex_zip)
city_county = targetedGeos.Geo.str.replace('DMA','').str.extract(regex_city_county)

In [43]:
geo.state.nunique()

57

In [44]:
geo.zip.nunique()

3909

In [25]:
targetedGeos.Geo.str.replace('DMA','')[:10]

0       Detroit, Michigan 
1            Toledo, Ohio 
2    Nashville, Tennessee 
3        Atlanta, Georgia 
4           36587, Alabama
5           36608, Alabama
6       39466, Mississippi
7       39503, Mississippi
8       39532, Mississippi
9       39564, Mississippi
Name: Geo, dtype: object

In [56]:
numberList = list()

In [34]:
def hasNumbers(inputString):
     return bool(re.search(r'\d', inputString))

In [57]:
for string in targetedGeos.Geo:
      numberList.append(hasNumbers(string))

In [59]:
sum(numberList) #this is counting non-uniques

18588

In [128]:
allOrders['CPV'].value_counts()

0.0027    50
0.0031    43
0.0026    39
0.0024    32
0.0030    30
0.0025    29
0.0023    24
0.0029    22
0.0034    19
0.0033    18
0.0028    18
0.0022    18
0.0021    16
0.0037    15
0.0032    13
0.0041    13
0.0020    11
0.0035    10
0.0043     9
0.0040     8
0.0039     7
0.0069     7
0.0044     6
0.0042     5
0.0046     5
0.0059     5
0.0055     5
0.0075     5
0.0057     5
0.0019     4
0.0058     4
0.0038     4
0.0063     3
0.0054     3
0.0045     3
0.0047     3
0.0048     3
0.0052     3
0.0051     3
0.0049     3
0.0062     3
0.0017     2
0.0018     2
0.0036     2
0.0066     2
0.0071     1
0.0050     1
0.0064     1
0.0074     1
0.0015     1
0.0068     1
0.0056     1
0.0067     1
0.0061     1
Name: CPV, dtype: int64

In [43]:
allOrders['CPV'].unique()

array([0.0021, 0.0027, 0.0025, 0.0032, 0.0057, 0.0028, 0.003 , 0.0023,
       0.0022, 0.0029, 0.0024, 0.0026, 0.0075, 0.0031, 0.0069, 0.0033,
       0.0044, 0.0047, 0.0045, 0.0042, 0.0059, 0.0054, 0.0051, 0.0048,
       0.0038, 0.0035, 0.0034, 0.0036, 0.0058, 0.002 , 0.0018, 0.0019,
       0.0017, 0.0041, 0.004 , 0.0043, 0.0046, 0.0055, 0.0037, 0.0039,
       0.0063, 0.0056, 0.0062, 0.0066, 0.0052, 0.0071, 0.0074, 0.0061,
       0.005 , 0.0068, 0.0064, 0.0067, 0.0015, 0.0049])

In [46]:
len(allOrders['CPV'].unique())

54

In [47]:
min(allOrders['CPV'])

0.0015

In [48]:
max(allOrders['CPV'])

0.0075

In [124]:
#Next steps:
#1 cpv digits
#2 location hierarchy build - determine if this should be by account as lead 
#3 H mod build

In [135]:
allData.dtypes

StartDate                datetime64[ns]
EndDate                  datetime64[ns]
CampaignBudget                  float64
SightlySpend                    float64
CPV                             float64
Objective                        object
AdvertiserSubCategory            object
Device                           object
AgeGroup                         object
Gender                           object
Geo                              object
HHI                              object
ParentalStatus                   object
dtype: object

In [147]:
#let's try and get stats around cpv by device
allData.columns

Index(['StartDate', 'EndDate', 'CampaignBudget', 'SightlySpend', 'CPV',
       'Objective', 'AdvertiserSubCategory', 'Device', 'AgeGroup', 'Gender',
       'Geo', 'HHI', 'ParentalStatus'],
      dtype='object')

In [152]:
#allOrders.merge(targetedDevices,on=)
allData.head()

StartDate    EndDate  \
AccountName OrderId                                                      
AMG         00FE3186-F3E0-48E2-999A-A893016D7517 2018-02-26 2018-05-19   
            00FE3186-F3E0-48E2-999A-A893016D7517 2018-02-26 2018-05-19   
            00FE3186-F3E0-48E2-999A-A893016D7517 2018-02-26 2018-05-19   
            00FE3186-F3E0-48E2-999A-A893016D7517 2018-02-26 2018-05-19   
            00FE3186-F3E0-48E2-999A-A893016D7517 2018-02-26 2018-05-19   

                                                  CampaignBudget  \
AccountName OrderId                                                
AMG         00FE3186-F3E0-48E2-999A-A893016D7517          6000.0   
            00FE3186-F3E0-48E2-999A-A893016D7517          6000.0   
            00FE3186-F3E0-48E2-999A-A893016D7517          6000.0   
            00FE3186-F3E0-48E2-999A-A893016D7517          6000.0   
            00FE3186-F3E0-48E2-999A-A893016D7517          6000.0   

                                                  SightlySpend     CPV  \
AccountName OrderId                                                      
AMG         00FE3186-F3E0-48E2-999A-A893016D7517        2475.0  0.0024   
            00FE3186-F3E0-48E2-999A-A893016D7517        2475.0  0.0024   
            00FE3186-F3E0-48E2-999A-A893016D7517        2475.0  0.0024   
            00FE3186-F3E0-48E2-999A-A893016D7517        2475.0  0.0024   
            00FE3186-F3E0-48E2-999A-A893016D7517        2475.0  0.0024   

                                                    Objective  \
AccountName OrderId                                             
AMG         00FE3186-F3E0-48E2-999A-A893016D7517  Full Funnel   
            00FE3186-F3E0-48E2-999A-A893016D7517  Full Funnel   
            00FE3186-F3E0-48E2-999A-A893016D7517  Full Funnel   
            00FE3186-F3E0-48E2-999A-A893016D7517  Full Funnel   
            00FE3186-F3E0-48E2-999A-A893016D7517  Full Funnel   

                                                         AdvertiserSubCategory  \
AccountName OrderId                                                              
AMG         00FE3186-F3E0-48E2-999A-A893016D7517  Lawyers: Personal Injury Law   
            00FE3186-F3E0-48E2-999A-A893016D7517  Lawyers: Personal Injury Law   
            00FE3186-F3E0-48E2-999A-A893016D7517  Lawyers: Personal Injury Law   
            00FE3186-F3E0-48E2-999A-A893016D7517  Lawyers: Personal Injury Law   
            00FE3186-F3E0-48E2-999A-A893016D7517  Lawyers: Personal Injury Law   

                                                                             Device  \
AccountName OrderId                                                                   
AMG         00FE3186-F3E0-48E2-999A-A893016D7517  Mobile devices with full browsers   
            00FE3186-F3E0-48E2-999A-A893016D7517  Mobile devices with full browsers   
            00FE3186-F3E0-48E2-999A-A893016D7517  Mobile devices with full browsers   
            00FE3186-F3E0-48E2-999A-A893016D7517  Mobile devices with full browsers   
            00FE3186-F3E0-48E2-999A-A893016D7517  Mobile devices with full browsers   

                                                 AgeGroup Gender  \
AccountName OrderId                                                
AMG         00FE3186-F3E0-48E2-999A-A893016D7517    35-44   Male   
            00FE3186-F3E0-48E2-999A-A893016D7517    35-44   Male   
            00FE3186-F3E0-48E2-999A-A893016D7517    35-44   Male   
            00FE3186-F3E0-48E2-999A-A893016D7517    35-44   Male   
            00FE3186-F3E0-48E2-999A-A893016D7517    35-44   Male   

                                                                       Geo  \
AccountName OrderId                                                          
AMG         00FE3186-F3E0-48E2-999A-A893016D7517  Nashville, Tennessee DMA   
            00FE3186-F3E0-48E2-999A-A893016D7517  Nashville, Tennessee DMA   
            00FE3186-F3E0-48E2-999A-A893016D7517  Nashville, Tennessee DMA   
            00FE3186-F3E0-4

In [163]:
#below code can separate geos into usable variables...state/dma/city/zip

/Users/murraywebb/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [20]:
regex
addr.str.replace('.', '').str.extract(regex)

NameError: name 'addr' is not defined

In [ ]:

# One or more letters
 # 2 capital letters
# Optional 4-digit extension

         city state         zip
0  Washington    DC       20003
1    Brooklyn    NY  11211-1755
2       Omaha    NE       68154
3  Pittsburgh    PA       15211
